<a href="https://colab.research.google.com/github/rajni-arora/Deep-Learning-Projects/blob/main/Topic_Modelling(Binary_Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 1: Importing dependencies

In [67]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [68]:
!pip install bert-for-tf2
!pip install sentencepiece

In [69]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

# Stage 2: Data preprocessing

## Loading files

We import files from our personal Google drive.

In [102]:
!wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

y
URL transformed to HTTPS due to an HSTS policy
--2021-10-31 09:55:05--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip.2’

trainingandtestdata 100%[===================>]  77.59M  16.9MB/s    in 5.6s    

2021-10-31 09:55:11 (13.8 MB/s) - ‘trainingandtestdata.zip.2’ saved [81363704/81363704]



In [103]:

!unzip '/content/trainingandtestdata.zip'

Archive:  /content/trainingandtestdata.zip
replace testdata.manual.2009.06.14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: testdata.manual.2009.06.14.csv  
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: training.1600000.processed.noemoticon.csv  


In [104]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/training.1600000.processed.noemoticon.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [105]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [106]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [107]:
data

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [108]:
data = pd.DataFrame(data)

In [109]:
data

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   text       1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [111]:
data.drop(data.index[:1598000], inplace=True)

data

,sentiment,text
1598000,4,watching David Cook on Larry King Live.. Didn'...
1598001,4,Sorting through seminar photos. Sifu's hands a...
1598002,4,@tonyfernandes congrats air asia welcome to t...
1598003,4,@suki1222 oh so good luck to you! i'm from gre...
1598004,4,Printed wikipedia is oh-so-handy http://bit.ly...
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [113]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1598000 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  2000 non-null   int64 
 1   text       2000 non-null   object
dtypes: int64(1), object(1)
memory usage: 46.9+ KB


In [82]:
data

,sentiment,text
1598000,4,watching David Cook on Larry King Live.. Didn'...
1598001,4,Sorting through seminar photos. Sifu's hands a...
1598002,4,@tonyfernandes congrats air asia welcome to t...
1598003,4,@suki1222 oh so good luck to you! i'm from gre...
1598004,4,Printed wikipedia is oh-so-handy http://bit.ly...
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [114]:
data.dropna(inplace=True)

In [115]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1598000 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  2000 non-null   int64 
 1   text       2000 non-null   object
dtypes: int64(1), object(1)
memory usage: 46.9+ KB


In [116]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Delete the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Delete URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Just keep letters and important punctuation
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Remove additional spaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [117]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [118]:
data_clean

["watching David Cook on Larry King Live.. Didn't get to see it 'coz i was at school... ",
 "Sorting through seminar photos. Sifu's hands are blurry in every one ",
 ' congrats air asia welcome to the fleet.',
 " oh so good luck to you! i'm from greece! and you? ",
 'Printed wikipedia is oh so handy via a gupta ',
 ' ur makin me nostalgic here despite the fact i was pretty young in the late s early s i hv very fond memories!',
 ' sunny day in Prescot is that not an oxymoron ',
 ' Either that or die laughing upon our first face to face.. ',
 ' Haha! well your head is very luckyy! x',
 'is eating a snickers ',
 ' See YOU there my friend.. I cant wait ',
 "Just trying to decide which piece of jewellery to spend the money on which I'll get from selling my revision guides on eBay ",
 " your webpage link doesn't work. interested in reading more! ",
 'home goint through my RSS and the off to get my weekly WoW dose.',
 ' I have the new cd All song are amaziinngg!! The duo with miley ROCK every

In [119]:
# Python code to convert list
# of string into list of list
#Nested List of data_clean
# List initialization
Input =data_clean
		
temp = []

# Getting elem in list of list format
for elem in Input:
	temp2 = elem.split(', ')
	temp.append((temp2))

# List initialization
Output = []

# Using Iteration to convert
# element into list of list
for elem in temp:
	temp3 = []
	for elem2 in elem:
		temp3.append(elem2)
	Output.append(temp3)

# printing
print(Output)


[["watching David Cook on Larry King Live.. Didn't get to see it 'coz i was at school... "], ["Sorting through seminar photos. Sifu's hands are blurry in every one "], [' congrats air asia welcome to the fleet.'], [" oh so good luck to you! i'm from greece! and you? "], ['Printed wikipedia is oh so handy via a gupta '], [' ur makin me nostalgic here despite the fact i was pretty young in the late s early s i hv very fond memories!'], [' sunny day in Prescot is that not an oxymoron '], [' Either that or die laughing upon our first face to face.. '], [' Haha! well your head is very luckyy! x'], ['is eating a snickers '], [' See YOU there my friend.. I cant wait '], ["Just trying to decide which piece of jewellery to spend the money on which I'll get from selling my revision guides on eBay "], [" your webpage link doesn't work. interested in reading more! "], ['home goint through my RSS and the off to get my weekly WoW dose.'], [' I have the new cd All song are amaziinngg!! The duo with m

In [120]:
print(data)

         sentiment                                               text
1598000          4  watching David Cook on Larry King Live.. Didn'...
1598001          4  Sorting through seminar photos. Sifu's hands a...
1598002          4  @tonyfernandes congrats air asia  welcome to t...
1598003          4  @suki1222 oh so good luck to you! i'm from gre...
1598004          4  Printed wikipedia is oh-so-handy http://bit.ly...
...            ...                                                ...
1599995          4  Just woke up. Having no school is the best fee...
1599996          4  TheWDB.com - Very cool to hear old Walt interv...
1599997          4  Are you ready for your MoJo Makeover? Ask me f...
1599998          4  Happy 38th Birthday to my boo of alll time!!! ...
1599999          4  happy #charitytuesday @theNSPCC @SparksCharity...

[2000 rows x 2 columns]


In [121]:
import pandas as pd
data1 = Output
df = pd.DataFrame(data1,columns=['data_clean'])
print(df)

                                             data_clean
0     watching David Cook on Larry King Live.. Didn'...
1     Sorting through seminar photos. Sifu's hands a...
2               congrats air asia welcome to the fleet.
3      oh so good luck to you! i'm from greece! and ...
4         Printed wikipedia is oh so handy via a gupta 
...                                                 ...
1995  Just woke up. Having no school is the best fee...
1996  TheWDB.com Very cool to hear old Walt intervie...
1997  Are you ready for your MoJo Makeover? Ask me f...
1998  Happy th Birthday to my boo of alll time!!! Tu...
1999                              happy charitytuesday 

[2000 rows x 1 columns]


In [122]:

extracted_col = data["sentiment"]
print("column to added from first dataframe to second:")
display(extracted_col)

df2 = df.join(extracted_col)
print("Second dataframe after adding column from first dataframe:")
display(df2)

column to added from first dataframe to second:


1598000    4
1598001    4
1598002    4
1598003    4
1598004    4
          ..
1599995    4
1599996    4
1599997    4
1599998    4
1599999    4
Name: sentiment, Length: 2000, dtype: int64

Second dataframe after adding column from first dataframe:


,data_clean,sentiment
0,watching David Cook on Larry King Live.. Didn'...,NaN
1,Sorting through seminar photos. Sifu's hands a...,NaN
2,congrats air asia welcome to the fleet.,NaN
3,oh so good luck to you! i'm from greece! and ...,NaN
4,Printed wikipedia is oh so handy via a gupta,NaN
...,...,...
1995,Just woke up. Having no school is the best fee...,NaN
1996,TheWDB.com Very cool to hear old Walt intervie...,NaN
1997,Are you ready for your MoJo Makeover? Ask me f...,NaN
1998,Happy th Birthday to my boo of alll time!!! Tu...,NaN


In [19]:
sent_of_train=[]
for sent in df2["data_clean"]:
    sent_of_train.append(sent.split())

In [20]:
sent_of_train[:1]

[['Awww',
  "that's",
  'a',
  'bummer.',
  'You',
  'shoulda',
  'got',
  'David',
  'Carr',
  'of',
  'Third',
  'Day',
  'to',
  'do',
  'it.',
  'D']]

In [21]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [22]:
model = Word2Vec(sent_of_train, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [23]:
model.wv.vocab

{'Awww': <gensim.models.keyedvectors.Vocab at 0x7f62cc1f2c50>,
 "that's": <gensim.models.keyedvectors.Vocab at 0x7f62c4549b50>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7f62c4549bd0>,
 'bummer.': <gensim.models.keyedvectors.Vocab at 0x7f62cc1ec490>,
 'You': <gensim.models.keyedvectors.Vocab at 0x7f62c4555490>,
 'shoulda': <gensim.models.keyedvectors.Vocab at 0x7f62c4555b50>,
 'got': <gensim.models.keyedvectors.Vocab at 0x7f62c4555a10>,
 'David': <gensim.models.keyedvectors.Vocab at 0x7f62c4555910>,
 'Carr': <gensim.models.keyedvectors.Vocab at 0x7f62cc1ec810>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7f62c4555b10>,
 'Third': <gensim.models.keyedvectors.Vocab at 0x7f62c4555650>,
 'Day': <gensim.models.keyedvectors.Vocab at 0x7f62c4555f50>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7f62c4555cd0>,
 'do': <gensim.models.keyedvectors.Vocab at 0x7f62c45557d0>,
 'it.': <gensim.models.keyedvectors.Vocab at 0x7f62c4555c90>,
 'D': <gensim.models.keyedvectors.Vocab at 0x7f62c4555e50>

In [24]:
model.most_similar('worried')[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('concerned', 0.9150667786598206),
 ('upset', 0.8812676668167114),
 ('worrying', 0.8764162659645081),
 ('Worried', 0.8705303072929382),
 ('heartbroken', 0.8704910278320312),
 ('nervous', 0.8689670562744141),
 ('depressed', 0.866463303565979),
 ('relieved', 0.8658481240272522),
 ('ecstatic', 0.8551777005195618),
 ('bragging', 0.855111300945282)]

In [25]:
model.most_similar('breast')[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('farts', 0.8849884271621704),
 ('liver', 0.8584917783737183),
 ('pox', 0.8584885597229004),
 ('poppy', 0.8556861281394958),
 ('lays', 0.8547933101654053),
 ('paw', 0.8537248969078064),
 ('cabbage', 0.8535439372062683),
 ('squishy', 0.851270854473114),
 ('hotdog', 0.8505198359489441),
 ('spoon', 0.8483153581619263)]

In [ ]:
## For Most Similar Bi-grams
bigram = Phrases(sent_of_train, min_count=1, threshold=2)

In [ ]:
bigram

In [ ]:
model = Word2Vec(bigram[sent_of_train], size=50, min_count=3, iter=20)

In [27]:
model.wv.most_similar("parrot") # Not Useful Approach

[('chihuahua', 0.937004566192627),
 ('rhythm', 0.9341604709625244),
 ('freddy', 0.9319455623626709),
 ('chipmunk', 0.9316178560256958),
 ('cub', 0.9308310747146606),
 ('kitteh', 0.9302920699119568),
 ('slurpee', 0.930044412612915),
 ('beagle', 0.9290580749511719),
 ('ladybug', 0.9284250140190125),
 ('tramp', 0.9279661178588867)]

In [28]:
!pip install top2vec

     |████████████████████████████████| 86 kB 4.0 MB/s 
     |████████████████████████████████| 15.7 MB 58 kB/s 
     |████████████████████████████████| 6.4 MB 42.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 34.3 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311934 sha256=f83dd9bd2b43442371751e8b5098c6a12ef453085939d7fa0b9e278c64f5bda8
  Stored in directory: /root/.cache/pip/wheels/73/5f/2f/9a259b84003b84847c259779206acecabb25ab56f1506ee72b
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82709 sha256=247c495ec6d6d44b59c91fd01bd9f6df5aef09adb7c18553886b8f3f1e3c1e6e
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52603 sha256=f2ff047375a4

Topic Modelling on Sentiment Dataset(Binary Dataset)

In [30]:
import ipywidgets as widgets
from IPython.display import clear_output, display
from top2vec import Top2Vec

In [ ]:
top2vec = Top2Vec(documents=df2["data_clean"].values, speed="learn", workers=4)

2021-10-31 04:45:03,110 - top2vec - INFO - Pre-processing documents for training
2021-10-31 04:46:11,394 - top2vec - INFO - Creating joint document/word embedding
2021-10-31 07:28:07,506 - top2vec - INFO - Creating lower dimension embedding of documents
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


# **Finding Similar Words**

In [ ]:
keywords_select_sw = widgets.Label('Enter keywords seperated by space: ')
display(keywords_select_sw)

keywords_input_sw = widgets.Text()
display(keywords_input_sw)

keywords_neg_select_sw = widgets.Label('Enter negative keywords seperated by space: ')
display(keywords_neg_select_sw)

keywords_neg_input_sw = widgets.Text()
display(keywords_neg_input_sw)


doc_num_select_sw = widgets.Label('Choose number of words: ')
display(doc_num_select_sw)

doc_num_input_sw = widgets.Text(value='20')
display(doc_num_input_sw)

def display_similar_words(*args):
    
    clear_output()
    display(keywords_select_sw)
    display(keywords_input_sw)
    display(keywords_neg_select_sw)
    display(keywords_neg_input_sw)
    display(doc_num_select_sw)
    display(doc_num_input_sw)
    display(sim_word_btn_sw)
    
    try:            
        words, word_scores = top2vec.similar_words(keywords=keywords_input_sw.value.split(), keywords_neg=keywords_neg_input_sw.value.split(), num_words=int(doc_num_input_sw.value))
        for word, score in zip(words, word_scores):
            print(f"{word} {score}")
   
    except Exception as e:
        print(e)
        
sim_word_btn_sw = widgets.Button(description="show similar words")
display(sim_word_btn_sw)
sim_word_btn_sw.on_click(display_similar_words)

# **Topic Search By Keywords**

In [ ]:
keywords_select_st = widgets.Label('Enter keywords seperated by space: ')
display(keywords_select_st)

keywords_input_st = widgets.Text()
display(keywords_input_st)

keywords_neg_select_st = widgets.Label('Enter negative keywords seperated by space: ')
display(keywords_neg_select_st)

keywords_neg_input_st = widgets.Text()
display(keywords_neg_input_st)

doc_num_select_st = widgets.Label('Choose number of topics: ')
display(doc_num_select_st)

doc_num_input_st = widgets.Text(value='5')
display(doc_num_input_st)

def display_similar_topics(*args):
    
    clear_output()
    display(keywords_select_st)
    display(keywords_input_st)
    display(keywords_neg_select_st)
    display(keywords_neg_input_st)
    display(doc_num_select_st)
    display(doc_num_input_st)
    display(keyword_btn_st)
    
    try:
        topic_words, word_scores, topic_scores, topic_nums = top2vec.search_topics(keywords=keywords_input_st.value.split(),num_topics=int(doc_num_input_st.value), keywords_neg=keywords_neg_input_st.value.split())
        for topic in topic_nums:
            top2vec.generate_topic_wordcloud(topic, background_color="black")
        
    except Exception as e:
        print(e)
        
keyword_btn_st = widgets.Button(description="show topics")
display(keyword_btn_st)
keyword_btn_st.on_click(display_similar_topics)

# **Using KeyBert**

In [ ]:
!pip install keybert[all]

In [ ]:
# converting ;Cleaned_Text' column into list
a = list(data['Cleaned_Text'])
  
# converting list into string and then joining it with space
b = ' '.join(str(e) for e in a)

In [ ]:
print(b[2000000:])

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,10]
plt.rcParams['figure.dpi'] = 80

In [ ]:
word_cloud = WordCloud(max_font_size=100).generate(b)
plt.imshow(word_cloud)
plt.axis('off')
plt.show()

In [ ]:
# kw_model.extract_keywords(b, keyphrase_ngram_range=(3, 3), stop_words='english', 
#                               use_maxsum=True, nr_candidates=20, top_n=5)
# Not able to run this code snippet for refined results, due to less computational resources

In [ ]:
kw_model.extract_keywords(b, keyphrase_ngram_range=(1, 4), stop_words="english")

In [ ]:
kw_model.extract_keywords(b, keyphrase_ngram_range=(1, 3), stop_words="english")